In [1]:
import sqlite3
con=sqlite3.connect('c:\\sqlite3\\contact_management_system.db')
cur=con.cursor()

# create tables

In [ ]:
cur.execute("CREATE TABLE CONTACT(fname text,\
lname text,\
email text,\
contact number,\
city text);")

In [ ]:
cur.execute("create table insert_log(\
            fname text,\
            lname text,\
            contact number,\
            email text,\
            city text,\
            insert_time text);")

cur.execute("create table update_log(\
            old_fname text,\
            new_fname text,\
            old_lname text,\
            new_lname text,\
            old_contact number,\
            new_contact number,\
            old_email text,\
            new_email text,\
            old_city text,\
            new_city text,\
            update_time text);")

cur.execute("create table delete_log(\
            fname text,\
            lname text,\
            contact number,\
            email text,\
            city text,\
            delete_time text);")

# create trigger for validate email

In [ ]:
cur.execute("CREATE TRIGGER IF NOT EXISTS validate_email_before_insert\
   BEFORE INSERT ON CONTACT\
   BEGIN\
   SELECT\
      CASE\
          WHEN NEW.email NOT LIKE '%@%.%' THEN\
          RAISE(ABORT,'Invalid email address')\
      END;\
    END;")

# create trigger for maintain log for insert,update and delete data information

In [ ]:
cur.execute("CREATE TRIGGER IF NOT EXISTS insert_log AFTER INSERT ON CONTACT\
            BEGIN\
                INSERT INTO insert_log VALUES (new.fname,new.lname,new.email,new.contact,new.city,datetime('now'));\
            END;")

In [ ]:
cur.execute("CREATE TRIGGER IF NOT EXISTS update_log AFTER UPDATE ON CONTACT\
            BEGIN\
                INSERT INTO update_log VALUES\
                (old.fname,new.fname,old.lname,new.lname,old.contact,new.contact,old.email,new.email,old.city,new.city,datetime('now'));\
            END;")

In [ ]:
cur.execute("CREATE TRIGGER IF NOT EXISTS delete_log AFTER DELETE ON CONTACT\
            BEGIN\
                INSERT INTO delete_log VALUES (old.fname,old.lname,old.contact,old.email,old.city,datetime('now'));\
            END;")

# record successfully instert because of correct email formate

In [2]:
cur.execute("INSERT INTO CONTACT VALUES('GORI','RADHA','radhag@kanha.com',9900099000,'SURAT'),\
('DAYAL','DEV','devdayal@gmail.com',9909099090,'DWARKA'),\
('DAYA','RAM','dayaram@yahoo.com',9109091090,'MATHURA')")

# it shows error because of wrong email formate

In [3]:
cur.execute("INSERT INTO CONTACT VALUES('VISHV','NATH','vnath1@gmailcom',8909089090,'VRINDAVAN')")

IntegrityError: Invalid email address

# printing all inserted records

In [4]:
cur.execute("SELECT * FROM CONTACT")
r=cur.fetchall()
h=['fname','lname','email','contact','city']
print(f'{h[0]: <10}{h[1]: <10}{h[2]: <20}{h[3]: <15}{h[4]}')
for i in r:
    print(f'{i[0]: <10}{i[1]: <10}{i[2]: <20}{i[3]: <15}{i[4]}')

fname     lname     email               contact        city
GORI      RADHA     radhag@kanha.com    9900099000     SURAT
DAYAL     DEV       devdayal@gmail.com  9909099090     DWARKA
DAYA      RAM       dayaram@yahoo.com   9109091090     MATHURA


# records in inser_log table

In [5]:
cur.execute("SELECT * FROM insert_log")
record=cur.fetchall()
h=['fname','lname','email','contact','city','insert_time']
print(f'{h[0]: <10}{h[1]: <10}{h[2]: <20}{h[3]: <15}{h[4]: <10}{h[5]}')
for i in record:
    print(f'{i[0]: <10}{i[1]: <10}{i[2]: <20}{i[3]: <15}{i[4]: <10}{i[5]}')

fname     lname     email               contact        city      insert_time
GORI      RADHA     radhag@kanha.com    9900099000     SURAT     2023-08-13 10:51:03
DAYAL     DEV       devdayal@gmail.com  9909099090     DWARKA    2023-08-13 10:51:03
DAYA      RAM       dayaram@yahoo.com   9109091090     MATHURA   2023-08-13 10:51:03


# update 1 record

In [6]:
cur.execute("UPDATE CONTACT SET city='GOKUL' WHERE fname='GORI'")

# records in update_log table

In [7]:
cur.execute("SELECT * FROM update_log")
record=cur.fetchall()
h=['oldfname','newfname','oldlname','newlname','oldcontact','newcontact','oldemail','newemail','oldcity','newcity','update_time']
print(f'{h[0]: <9}{h[1]: <9}{h[2]: <9}{h[3]: <9}{h[4]: <11}{h[5]: <11}{h[6]: <17}{h[7]: <17}{h[8]: <8}{h[9]: <8}{h[10]}')
for i in record:
    print(f'{i[0]: <9}{i[1]: <9}{i[2]: <9}{i[3]: <9}{i[4]: <11}{i[5]: <11}{i[6]: <17}{i[7]: <17}{i[8]: <8}{i[9]: <8}{i[10]}')

oldfname newfname oldlname newlname oldcontact newcontact oldemail         newemail         oldcity newcity update_time
GORI     GORI     RADHA    RADHA    9900099000 9900099000 radhag@kanha.com radhag@kanha.com SURAT   GOKUL   2023-08-13 10:51:15


# delete 1 record

In [8]:
cur.execute("DELETE FROM CONTACT WHERE fname='DAYA'")

# records in delete_log table

In [9]:
cur.execute("SELECT * FROM delete_log")
record=cur.fetchall()
h=['fname','lname','contact','email','city','delete_time']
print(f'{h[0]: <10}{h[1]: <10}{h[2]: <15}{h[3]: <20}{h[4]: <10}{h[5]}')
for i in record:
    print(f'{i[0]: <10}{i[1]: <10}{i[2]: <15}{i[3]: <20}{i[4]: <10}{i[5]}')

fname     lname     contact        email               city      delete_time
DAYA      RAM       9109091090     dayaram@yahoo.com   MATHURA   2023-08-13 10:51:21


In [10]:
con.commit()

In [11]:
con.close()